# Character-Level LSTM in PyTorch

In this notebook, I'll construct a character-level LSTM with PyTorch. The network will train character by character on some text, then generate new text character by character. As an example, I will train on Anna Karenina. **This model will be able to generate new text based on the text from the book!**

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Below is the general architecture of the character-wise RNN.

<img src="https://github.com/udacity/deep-learning-v2-pytorch/raw/3a95d118f9df5a86826e1791c5c100817f0fd924/recurrent-neural-networks/char-rnn/assets/charseq.jpeg" width="500">

First let's load in our required resources for data loading and model creation.

In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

## Load in Data

Then, we'll load the Anna Karenina text file and convert it into integers for our network to use. 

In [2]:
# open text file and read in data as `text`
with open('data/anna.txt', 'r') as f:
    text = f.read()

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

### Tokenization

In the cells, below, I'm creating a couple **dictionaries** to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [4]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

And we can see those same characters from above, encoded as integers.

In [5]:
encoded[:100]

array([23, 41, 68, 61, 20, 66, 80, 58, 52, 65, 65, 65, 55, 68, 61, 61, 34,
       58,  7, 68, 42, 75, 78, 75, 66, 47, 58, 68, 80, 66, 58, 68, 78, 78,
       58, 68, 78, 75,  9, 66, 21, 58, 66,  8, 66, 80, 34, 58, 56, 63, 41,
       68, 61, 61, 34, 58,  7, 68, 42, 75, 78, 34, 58, 75, 47, 58, 56, 63,
       41, 68, 61, 61, 34, 58, 75, 63, 58, 75, 20, 47, 58, 57, 36, 63, 65,
       36, 68, 34, 12, 65, 65, 29,  8, 66, 80, 34, 20, 41, 75, 63])

## Pre-processing the data

As you can see in our char-RNN image above, our LSTM expects an input that is **one-hot encoded** meaning that each character is converted into an integer (via our created dictionary) and *then* converted into a column vector where only it's corresponding integer index will have the value of 1 and the rest of the vector will be filled with 0's. Since we're one-hot encoding the data, let's make a function to do that!


In [6]:
def one_hot_encode(arr, n_labels):
    """Returns a one-hot-encoded matrix of shape (len(arr), n_labels), where each row is a one_hot vector of the corresponding entry in arr"""
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [7]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


## Making training mini-batches


To train on this data, we also want to create mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="https://github.com/udacity/deep-learning-v2-pytorch/raw/3a95d118f9df5a86826e1791c5c100817f0fd924/recurrent-neural-networks/char-rnn/assets/sequence_batching@1x.png" width=500px>


<br>

In this example, we'll take the encoded characters (passed in as the `arr` parameter) and split them into multiple sequences, given by `batch_size`. Each of our sequences will be `seq_length` long.

### Creating Batches

**1. The first thing we need to do is discard some of the text so we only have completely full mini-batches. **

Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences in a batch) and $M$ is the seq_length or number of time steps in a sequence. Then, to get the total number of batches, $K$, that we can make from the array `arr`, you divide the length of `arr` by the number of characters per batch. Once you know the number of batches, you can get the total number of characters to keep from `arr`, $N * M * K$.

**2. After that, we need to split `arr` into $N$ batches. ** 

You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences in a batch, so let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$.

**3. Now that we have this array, we can iterate through it to get our mini-batches. **

The idea is each batch is a $N \times M$ window on the $N \times (M * K)$ array. For each subsequent batch, the window moves over by `seq_length`. We also want to create both the input and target arrays. Remember that the targets are just the inputs shifted over by one character. The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of tokens in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `seq_length` wide.

> **TODO:** Write the code for creating batches in the function below. The exercises in this notebook _will not be easy_. I've provided a notebook with solutions alongside this notebook. If you get stuck, checkout the solutions. The most important thing is that you don't copy and paste the code into here, **type out the solution code yourself.**

In [8]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    ## TODO: Get the number of batches we can make
    n_batches = len(arr) // (batch_size * seq_length)
    
    ## TODO: Keep only enough characters to make full batches
    charsToKeep = n_batches * batch_size * seq_length
    arr = arr[:charsToKeep]
    
    ## TODO: Reshape into batch_size rows
    arr = arr.reshape(batch_size, n_batches * seq_length)
    
    ## TODO: Iterate over the batches using a window of size seq_length
    for n in range(0, arr.shape[1], seq_length):
        # The features
        startIndex = n * seq_length
        endIndex = (n + 1) * seq_length

        x = arr[:, startIndex : endIndex]
        
        # The targets, shifted by one
        y = arr[:, startIndex + 1 : endIndex + 1]
        
        if x.size == 0 or x.size != y.size:
            return
        
        yield x, y


### Test Your Implementation

Now I'll make some data sets and we can check out what's going on as we batch data. Here, as an example, I'm going to use a batch size of 8 and 50 sequence steps.

In [9]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [10]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[23 41 68 61 20 66 80 58 52 65]
 [47 57 63 58 20 41 68 20 58 68]
 [66 63 48 58 57 80 58 68 58  7]
 [47 58 20 41 66 58 33 41 75 66]
 [58 47 68 36 58 41 66 80 58 20]
 [33 56 47 47 75 57 63 58 68 63]
 [58  2 63 63 68 58 41 68 48 58]
 [ 6 82 78 57 63 47  9 34 12 58]]

y
 [[41 68 61 20 66 80 58 52 65 65]
 [57 63 58 20 41 68 20 58 68 20]
 [63 48 58 57 80 58 68 58  7 57]
 [58 20 41 66 58 33 41 75 66  7]
 [47 68 36 58 41 66 80 58 20 66]
 [56 47 47 75 57 63 58 68 63 48]
 [ 2 63 63 68 58 41 68 48 58 47]
 [82 78 57 63 47  9 34 12 58 54]]


If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[25  8 60 11 45 27 28 73  1  2]
 [17  7 20 73 45  8 60 45 73 60]
 [27 20 80 73  7 28 73 60 73 65]
 [17 73 45  8 27 73 66  8 46 27]
 [73 17 60 12 73  8 27 28 73 45]
 [66 64 17 17 46  7 20 73 60 20]
 [73 76 20 20 60 73  8 60 80 73]
 [47 35 43  7 20 17 24 50 37 73]]

y
 [[ 8 60 11 45 27 28 73  1  2  2]
 [ 7 20 73 45  8 60 45 73 60 45]
 [20 80 73  7 28 73 60 73 65  7]
 [73 45  8 27 73 66  8 46 27 65]
 [17 60 12 73  8 27 28 73 45 27]
 [64 17 17 46  7 20 73 60 20 80]
 [76 20 20 60 73  8 60 80 73 17]
 [35 43  7 20 17 24 50 37 73 36]]
 ```
 although the exact numbers may be different. Check to make sure the data is shifted over one step for `y`.

---
## Defining the network with PyTorch

Below is where you'll define the network.

<img src="https://github.com/udacity/deep-learning-v2-pytorch/raw/3a95d118f9df5a86826e1791c5c100817f0fd924/recurrent-neural-networks/char-rnn/assets/charRNN.png" width=500px>

Next, you'll use PyTorch to define the architecture of the network. We start by defining the layers and operations we want. Then, define a method for the forward pass. You've also been given a method for predicting characters.

### Model Structure

In `__init__` the suggested structure is as follows:
* Create and store the necessary dictionaries (this has been done for you)
* Define an LSTM layer that takes as params: an input size (the number of characters), a hidden layer size `n_hidden`, a number of layers `n_layers`, a dropout probability `drop_prob`, and a batch_first boolean (True, since we are batching)
* Define a dropout layer with `drop_prob`
* Define a fully-connected layer with params: input size `n_hidden` and output size (the number of characters)
* Finally, initialize the weights (again, this has been given)

Note that some parameters have been named and given in the `__init__` function, and we use them and store them by doing something like `self.drop_prob = drop_prob`.

---
### LSTM Inputs/Outputs

You can create a basic [LSTM layer](https://pytorch.org/docs/stable/nn.html#lstm) as follows

```python
self.lstm = nn.LSTM(input_size, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
```

where `input_size` is the number of characters this cell expects to see as sequential input, and `n_hidden` is the number of units in the hidden layers in the cell. And we can add dropout by adding a dropout parameter with a specified probability; this will automatically add dropout to the inputs or outputs. Finally, in the `forward` function, we can stack up the LSTM cells into layers using `.view`. With this, you pass in a list of cells and it will send the output of one cell into the next cell.

We also need to create an initial hidden state of all zeros. This is done like so

```python
self.init_hidden()
```

In [11]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [12]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the layers of the model
        self.lstm = nn.LSTM(len(tokens), n_hidden, n_layers, dropout=drop_prob, batch_first=True)
        
        self.dropoutLayer = nn.Dropout(p=drop_prob)
        
        self.fullyConnected = nn.Linear(n_hidden, len(tokens))
      
      
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''

        ## TODO: Get the outputs and the new hidden state from the lstm
        out1, hidden = self.lstm(x, hidden)

        out2 = self.dropoutLayer(out1)
        out3 = self.fullyConnected(out2)

        flattened = torch.flatten(out3, end_dim=1)

        # return the final output and the hidden state
        return flattened, hidden
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

## Time to train

The train function gives us the ability to set the number of epochs, the learning rate, and other parameters.

Below we're using an Adam optimizer and cross entropy loss since we are looking at character class scores as output. We calculate the loss and perform backpropagation, as usual!

A couple of details about training: 
>* Within the batch loop, we detach the hidden state from its history; this time setting it equal to a new *tuple* variable because an LSTM has a hidden state that is a tuple of the hidden and cell states.
* We use [`clip_grad_norm_`](https://pytorch.org/docs/stable/_modules/torch/nn/utils/clip_grad.html) to help prevent exploding gradients.

In [13]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

## Instantiating the model

Now we can actually train the network. First we'll create the network itself, with some given hyperparameters. Then, define the mini-batches sizes, and start training!

In [14]:
## TODO: set your model hyperparameters
# define and print the net
n_hidden=20
n_layers=5

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(83, 20, num_layers=5, batch_first=True, dropout=0.5)
  (dropoutLayer): Dropout(p=0.5, inplace=False)
  (fullyConnected): Linear(in_features=20, out_features=83, bias=True)
)


### Set your training hyperparameters!

In [15]:
batch_size = 10
seq_length = 7
n_epochs =  5 #start small if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/5... Step: 10... Loss: 4.3621... Val Loss: 4.3852
Epoch: 1/5... Step: 20... Loss: 4.3314... Val Loss: 4.3345
Epoch: 1/5... Step: 30... Loss: 4.2449... Val Loss: 4.2467
Epoch: 1/5... Step: 40... Loss: 4.2028... Val Loss: 4.0736
Epoch: 1/5... Step: 50... Loss: 3.9202... Val Loss: 3.8520
Epoch: 1/5... Step: 60... Loss: 3.5099... Val Loss: 3.6231
Epoch: 1/5... Step: 70... Loss: 3.6279... Val Loss: 3.4484
Epoch: 1/5... Step: 80... Loss: 3.3952... Val Loss: 3.3395
Epoch: 1/5... Step: 90... Loss: 3.5778... Val Loss: 3.2695
Epoch: 1/5... Step: 100... Loss: 3.3518... Val Loss: 3.2283
Epoch: 1/5... Step: 110... Loss: 3.4770... Val Loss: 3.2012
Epoch: 1/5... Step: 120... Loss: 3.4224... Val Loss: 3.1847
Epoch: 1/5... Step: 130... Loss: 3.0669... Val Loss: 3.1745
Epoch: 1/5... Step: 140... Loss: 3.2149... Val Loss: 3.1665
Epoch: 1/5... Step: 150... Loss: 3.4224... Val Loss: 3.1607
Epoch: 1/5... Step: 160... Loss: 3.1156... Val Loss: 3.1569
Epoch: 1/5... Step: 170... Loss: 2.9864... Val Lo

Epoch: 1/5... Step: 1380... Loss: 3.0464... Val Loss: 3.1324
Epoch: 1/5... Step: 1390... Loss: 3.0964... Val Loss: 3.1324
Epoch: 1/5... Step: 1400... Loss: 3.1809... Val Loss: 3.1311
Epoch: 1/5... Step: 1410... Loss: 3.0412... Val Loss: 3.1306
Epoch: 1/5... Step: 1420... Loss: 3.1934... Val Loss: 3.1309
Epoch: 1/5... Step: 1430... Loss: 3.2024... Val Loss: 3.1315
Epoch: 1/5... Step: 1440... Loss: 3.3274... Val Loss: 3.1318
Epoch: 1/5... Step: 1450... Loss: 3.2511... Val Loss: 3.1314
Epoch: 1/5... Step: 1460... Loss: 3.0464... Val Loss: 3.1311
Epoch: 1/5... Step: 1470... Loss: 3.0791... Val Loss: 3.1307
Epoch: 1/5... Step: 1480... Loss: 3.2536... Val Loss: 3.1304
Epoch: 1/5... Step: 1490... Loss: 3.0477... Val Loss: 3.1306
Epoch: 1/5... Step: 1500... Loss: 3.3465... Val Loss: 3.1308
Epoch: 1/5... Step: 1510... Loss: 3.1747... Val Loss: 3.1305
Epoch: 1/5... Step: 1520... Loss: 3.1782... Val Loss: 3.1301
Epoch: 1/5... Step: 1530... Loss: 3.1234... Val Loss: 3.1301
Epoch: 1/5... Step: 1540

Epoch: 1/5... Step: 2730... Loss: 2.9491... Val Loss: 3.0060
Epoch: 1/5... Step: 2740... Loss: 2.9999... Val Loss: 3.0046
Epoch: 1/5... Step: 2750... Loss: 2.8171... Val Loss: 3.0058
Epoch: 1/5... Step: 2760... Loss: 2.9992... Val Loss: 3.0092
Epoch: 1/5... Step: 2770... Loss: 2.9015... Val Loss: 3.0085
Epoch: 1/5... Step: 2780... Loss: 2.9501... Val Loss: 3.0078
Epoch: 1/5... Step: 2790... Loss: 3.1368... Val Loss: 3.0098
Epoch: 1/5... Step: 2800... Loss: 2.9204... Val Loss: 3.0097
Epoch: 1/5... Step: 2810... Loss: 2.8790... Val Loss: 3.0052
Epoch: 1/5... Step: 2820... Loss: 2.7890... Val Loss: 3.0048
Epoch: 1/5... Step: 2830... Loss: 3.2890... Val Loss: 3.0036
Epoch: 1/5... Step: 2840... Loss: 3.1628... Val Loss: 3.0025
Epoch: 1/5... Step: 2850... Loss: 3.1169... Val Loss: 3.0004
Epoch: 1/5... Step: 2860... Loss: 3.0118... Val Loss: 3.0014
Epoch: 1/5... Step: 2870... Loss: 3.1396... Val Loss: 3.0019
Epoch: 1/5... Step: 2880... Loss: 3.1732... Val Loss: 3.0002
Epoch: 1/5... Step: 2890

Epoch: 2/5... Step: 4080... Loss: 2.7897... Val Loss: 2.9086
Epoch: 2/5... Step: 4090... Loss: 3.0247... Val Loss: 2.9094
Epoch: 2/5... Step: 4100... Loss: 2.7143... Val Loss: 2.9152
Epoch: 2/5... Step: 4110... Loss: 2.8315... Val Loss: 2.9123
Epoch: 2/5... Step: 4120... Loss: 2.8653... Val Loss: 2.9066
Epoch: 2/5... Step: 4130... Loss: 2.6477... Val Loss: 2.9028
Epoch: 2/5... Step: 4140... Loss: 3.0060... Val Loss: 2.9042
Epoch: 2/5... Step: 4150... Loss: 3.1832... Val Loss: 2.9058
Epoch: 2/5... Step: 4160... Loss: 3.2438... Val Loss: 2.9044
Epoch: 2/5... Step: 4170... Loss: 3.0670... Val Loss: 2.9037
Epoch: 2/5... Step: 4180... Loss: 2.9140... Val Loss: 2.9032
Epoch: 2/5... Step: 4190... Loss: 3.1990... Val Loss: 2.9008
Epoch: 2/5... Step: 4200... Loss: 2.8288... Val Loss: 2.8960
Epoch: 2/5... Step: 4210... Loss: 2.6816... Val Loss: 2.8964
Epoch: 2/5... Step: 4220... Loss: 3.1304... Val Loss: 2.8938
Epoch: 2/5... Step: 4230... Loss: 2.7466... Val Loss: 2.8964
Epoch: 2/5... Step: 4240

Epoch: 2/5... Step: 5430... Loss: 2.6559... Val Loss: 2.8107
Epoch: 2/5... Step: 5440... Loss: 2.8128... Val Loss: 2.8105
Epoch: 2/5... Step: 5450... Loss: 2.8258... Val Loss: 2.8099
Epoch: 2/5... Step: 5460... Loss: 2.6433... Val Loss: 2.8117
Epoch: 2/5... Step: 5470... Loss: 2.7849... Val Loss: 2.8186
Epoch: 2/5... Step: 5480... Loss: 2.9230... Val Loss: 2.8127
Epoch: 2/5... Step: 5490... Loss: 2.7708... Val Loss: 2.8128
Epoch: 2/5... Step: 5500... Loss: 3.0188... Val Loss: 2.8090
Epoch: 2/5... Step: 5510... Loss: 3.0158... Val Loss: 2.8099
Epoch: 2/5... Step: 5520... Loss: 2.5495... Val Loss: 2.8115
Epoch: 2/5... Step: 5530... Loss: 2.7867... Val Loss: 2.8068
Epoch: 2/5... Step: 5540... Loss: 3.0179... Val Loss: 2.8056
Epoch: 2/5... Step: 5550... Loss: 3.0237... Val Loss: 2.8096
Epoch: 2/5... Step: 5560... Loss: 3.0988... Val Loss: 2.8092
Epoch: 2/5... Step: 5570... Loss: 3.1217... Val Loss: 2.8063
Epoch: 2/5... Step: 5580... Loss: 2.7209... Val Loss: 2.8089
Epoch: 2/5... Step: 5590

...

Epoch: 4/5... Step: 12140... Loss: 2.7901... Val Loss: 2.7202
Epoch: 4/5... Step: 12150... Loss: 2.6413... Val Loss: 2.7195
Epoch: 4/5... Step: 12160... Loss: 2.9745... Val Loss: 2.7222
Epoch: 4/5... Step: 12170... Loss: 2.6514... Val Loss: 2.7207
Epoch: 4/5... Step: 12180... Loss: 2.7315... Val Loss: 2.7218
Epoch: 4/5... Step: 12190... Loss: 3.0681... Val Loss: 2.7217
Epoch: 4/5... Step: 12200... Loss: 2.9311... Val Loss: 2.7219
Epoch: 4/5... Step: 12210... Loss: 2.6608... Val Loss: 2.7240
Epoch: 4/5... Step: 12220... Loss: 3.0268... Val Loss: 2.7244
Epoch: 4/5... Step: 12230... Loss: 3.1290... Val Loss: 2.7227
Epoch: 4/5... Step: 12240... Loss: 2.9611... Val Loss: 2.7232
Epoch: 4/5... Step: 12250... Loss: 2.6892... Val Loss: 2.7233
Epoch: 4/5... Step: 12260... Loss: 2.9064... Val Loss: 2.7229
Epoch: 4/5... Step: 12270... Loss: 2.8216... Val Loss: 2.7224
Epoch: 4/5... Step: 12280... Loss: 3.0029... Val Loss: 2.7216
Epoch: 4/5... Step: 12290... Loss: 2.8820... Val Loss: 2.7214
Epoch: 4

Epoch: 4/5... Step: 13470... Loss: 2.4976... Val Loss: 2.7149
Epoch: 4/5... Step: 13480... Loss: 2.7017... Val Loss: 2.7140
Epoch: 4/5... Step: 13490... Loss: 2.8531... Val Loss: 2.7133
Epoch: 4/5... Step: 13500... Loss: 2.6816... Val Loss: 2.7124
Epoch: 4/5... Step: 13510... Loss: 2.8561... Val Loss: 2.7121
Epoch: 4/5... Step: 13520... Loss: 2.3179... Val Loss: 2.7135
Epoch: 4/5... Step: 13530... Loss: 2.7769... Val Loss: 2.7125
Epoch: 4/5... Step: 13540... Loss: 2.3084... Val Loss: 2.7119
Epoch: 4/5... Step: 13550... Loss: 2.7822... Val Loss: 2.7119
Epoch: 4/5... Step: 13560... Loss: 2.7852... Val Loss: 2.7129
Epoch: 4/5... Step: 13570... Loss: 2.9311... Val Loss: 2.7142
Epoch: 4/5... Step: 13580... Loss: 2.7231... Val Loss: 2.7139
Epoch: 4/5... Step: 13590... Loss: 2.8088... Val Loss: 2.7128
Epoch: 4/5... Step: 13600... Loss: 2.6784... Val Loss: 2.7138
Epoch: 4/5... Step: 13610... Loss: 2.7942... Val Loss: 2.7139
Epoch: 4/5... Step: 13620... Loss: 2.9496... Val Loss: 2.7126
Epoch: 4

Epoch: 5/5... Step: 14800... Loss: 2.8952... Val Loss: 2.7119
Epoch: 5/5... Step: 14810... Loss: 2.4727... Val Loss: 2.7084
Epoch: 5/5... Step: 14820... Loss: 2.5970... Val Loss: 2.7107
Epoch: 5/5... Step: 14830... Loss: 2.7214... Val Loss: 2.7172
Epoch: 5/5... Step: 14840... Loss: 2.5798... Val Loss: 2.7140
Epoch: 5/5... Step: 14850... Loss: 2.7420... Val Loss: 2.7089
Epoch: 5/5... Step: 14860... Loss: 2.5915... Val Loss: 2.7093
Epoch: 5/5... Step: 14870... Loss: 2.6082... Val Loss: 2.7103
Epoch: 5/5... Step: 14880... Loss: 2.8439... Val Loss: 2.7116
Epoch: 5/5... Step: 14890... Loss: 2.5766... Val Loss: 2.7120
Epoch: 5/5... Step: 14900... Loss: 2.8848... Val Loss: 2.7094
Epoch: 5/5... Step: 14910... Loss: 2.9448... Val Loss: 2.7085
Epoch: 5/5... Step: 14920... Loss: 2.9306... Val Loss: 2.7066
Epoch: 5/5... Step: 14930... Loss: 2.4869... Val Loss: 2.7070
Epoch: 5/5... Step: 14940... Loss: 2.6936... Val Loss: 2.7099
Epoch: 5/5... Step: 14950... Loss: 2.6990... Val Loss: 2.7121
Epoch: 5

Epoch: 5/5... Step: 16130... Loss: 2.6659... Val Loss: 2.7017
Epoch: 5/5... Step: 16140... Loss: 2.7714... Val Loss: 2.7011
Epoch: 5/5... Step: 16150... Loss: 2.7968... Val Loss: 2.7003
Epoch: 5/5... Step: 16160... Loss: 2.6964... Val Loss: 2.7017
Epoch: 5/5... Step: 16170... Loss: 2.8577... Val Loss: 2.7016
Epoch: 5/5... Step: 16180... Loss: 2.4939... Val Loss: 2.7000
Epoch: 5/5... Step: 16190... Loss: 2.9336... Val Loss: 2.6992
Epoch: 5/5... Step: 16200... Loss: 3.0022... Val Loss: 2.6994
Epoch: 5/5... Step: 16210... Loss: 2.5712... Val Loss: 2.6994
Epoch: 5/5... Step: 16220... Loss: 2.6541... Val Loss: 2.7021
Epoch: 5/5... Step: 16230... Loss: 2.7754... Val Loss: 2.7003
Epoch: 5/5... Step: 16240... Loss: 2.7295... Val Loss: 2.7006
Epoch: 5/5... Step: 16250... Loss: 2.8586... Val Loss: 2.7002
Epoch: 5/5... Step: 16260... Loss: 2.5720... Val Loss: 2.7010
Epoch: 5/5... Step: 16270... Loss: 2.7956... Val Loss: 2.7019
Epoch: 5/5... Step: 16280... Loss: 2.4389... Val Loss: 2.6997
Epoch: 5

Epoch: 5/5... Step: 17460... Loss: 3.0172... Val Loss: 2.7037
Epoch: 5/5... Step: 17470... Loss: 3.0602... Val Loss: 2.7018
Epoch: 5/5... Step: 17480... Loss: 2.7890... Val Loss: 2.7037
Epoch: 5/5... Step: 17490... Loss: 2.8813... Val Loss: 2.6999
Epoch: 5/5... Step: 17500... Loss: 2.9352... Val Loss: 2.6990
Epoch: 5/5... Step: 17510... Loss: 2.8661... Val Loss: 2.6993
Epoch: 5/5... Step: 17520... Loss: 2.6101... Val Loss: 2.6993
Epoch: 5/5... Step: 17530... Loss: 2.8406... Val Loss: 2.7003
Epoch: 5/5... Step: 17540... Loss: 2.6290... Val Loss: 2.7048
Epoch: 5/5... Step: 17550... Loss: 2.6901... Val Loss: 2.7056
Epoch: 5/5... Step: 17560... Loss: 2.8582... Val Loss: 2.6987
Epoch: 5/5... Step: 17570... Loss: 2.5478... Val Loss: 2.7005
Epoch: 5/5... Step: 17580... Loss: 2.8342... Val Loss: 2.7002
Epoch: 5/5... Step: 17590... Loss: 2.6584... Val Loss: 2.6996
Epoch: 5/5... Step: 17600... Loss: 2.8540... Val Loss: 2.6990
Epoch: 5/5... Step: 17610... Loss: 2.4478... Val Loss: 2.6991
Epoch: 5

## Getting the best model

To set your hyperparameters to get the best performance, you'll want to watch the training and validation losses. If your training loss is much lower than the validation loss, you're overfitting. Increase regularization (more dropout) or use a smaller network. If the training and validation losses are close, you're underfitting so you can increase the size of the network.

## Hyperparameters

Here are the hyperparameters for the network.

In defining the model:
* `n_hidden` - The number of units in the hidden layers.
* `n_layers` - Number of hidden LSTM layers to use.

We assume that dropout probability and learning rate will be kept at the default, in this example.

And in training:
* `batch_size` - Number of sequences running through the network in one pass.
* `seq_length` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lr` - Learning rate for training

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `n_hidden` and `n_layers`. I would advise that you always use `n_layers` of either 2/3. The `n_hidden` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `n_hidden` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

## Checkpoint

After training, we'll save the model so we can load it again later if we need too. Here I'm saving the parameters needed to create the same architecture, the hidden layer hyperparameters and the text characters.

In [16]:
# change the name, for saving multiple files
model_name = 'rnn_x_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

---
## Making Predictions

Now that the model is trained, we'll want to sample from it and make predictions about next characters! To sample, we pass in a character and have the network predict the next character. Then we take that character, pass it back in, and get another predicted character. Just keep doing this and you'll generate a bunch of text!

### A note on the `predict`  function

The output of our RNN is from a fully-connected layer and it outputs a **distribution of next-character scores**.

> To actually get the next character, we apply a softmax function, which gives us a *probability* distribution that we can then sample to predict the next character.

### Top K sampling

Our predictions come from a categorical probability distribution over all the possible characters. We can make the sample text and make it more reasonable to handle (with less variables) by only considering some $K$ most probable characters. This will prevent the network from giving us completely absurd characters while allowing it to introduce some noise and randomness into the sampled text. Read more about [topk, here](https://pytorch.org/docs/stable/torch.html#torch.topk).


In [17]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

### Priming and generating text 

Typically you'll want to prime the network so you can build up a hidden state. Otherwise the network will start out generating characters at random. In general the first bunch of characters will be a little rough since it hasn't built up a long history of characters to predict from.

In [18]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [19]:
print(sample(net, 1000, prime='Anna', top_k=5))

Annand hod to sist wes hheris shond soserarise tanted to se hhithe wot tinde hiti   Ang ande hind wis a at ting and ter setirin shate wen sas shhandi hir and ann winter wosent and and sir hodterir ang shon her altenl shoren wetarit ann annd a sonl winnte wosat ant ande sint tetan hat to wetin whisten angen as andeng thh atir to shet an thongar tet to ter he tindas tisin so thar tinn war hhare toet so hhatisi an ang teteringisendi ten wased tat ann saden atit hir thingand hhhi  "
"tes shandet tand tir and sand won toetite a soret he wen wong wodis wate toe he and he ang and,an a sadire sitis hhas hired hhhath he ande ald thor astinn andirtandasiserith annd to tes ter ser whote ann witerind at to thinn titinn asinnt watid harase tan tot tid ser ten ant ang sate wet a se hir atant,in hare he tat angit sen to a thh to a hat tentann hit tand wort antinn wan ther sot hen theetidere a  
Antit sesitherann at ant as ser he tester hont honlat sonl sasare as thetatedt ses tad tot wos he whhored t

## Loading a checkpoint

In [20]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_x_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [21]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))

And Levin saidaterinnlos ang whatinde hongor tes alg tit tar at han wen tit wot setit sare wet ant tin heed te al hhhe tan tar wate so wed atinte terhed to ang, an att ang thas and wong toit asete titithaten ten sat to wet hor and sath to se she att to wennetath al aterated wor hintetederin tit alged shor atedis at tidennd wond, waden aln thas toi  "A  ""

  toen sor the,iren hhis to aldete tidend whin test set a hhet wos he to wer tesannten ten honlir anned thoth wont toitindas al tas tet wiset west tarirand an tiran hore atense sonle tar a as torin her hore an tharisanted wos and sor has an hote a  

 A  


" 
thi toet hhasi theen to sese ten at hhe he a aln ann ter wenne heet thatar tesasire handar an tind tend ate had she as hond wodi  An hen ang al tit hon hisetent ante wot woder tin to ang tatede an was ang a to ten sont the, atangas te and sinda hhiten hirhedan an to sere anga the se he whet war algeder and to thin han hasar tet a  toet sirint tisas ann sor and whoshat hat hee h